In [29]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from pyntcloud import PyntCloud 

In [2]:
cap = cv2.VideoCapture('../../test.mp4')
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

U = np.zeros((n_frames, p0.shape[0]))
V = np.zeros((n_frames, p0.shape[0]))

S = np.ones((p0.shape[0],1))

U[0,:] = p0[:,0,0]
V[0,:] = p0[:,0,1]

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
loops = 0
good_features = [p0]

ret,frame = cap.read()

while frame is not None:
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
#     print(st.shape)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
#     print(good_new.shape)
    U[loops, :] = good_new[:, 0]
    V[loops, :] = good_new[:, 1]
    
    # draw the tracks
#     for i,(new,old) in enumerate(zip(good_new,good_old)):
#         a,b = new.ravel()
#         c,d = old.ravel()
#         mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
#         frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    
#     img = cv2.add(frame,mask)
#     cv2.imwrite("../output/frame_{}.png".format(loops), img)
    
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    good_features.append(p0)
    
    ret,frame = cap.read()
    loops += 1
    
cap.release()

In [3]:
frames = np.array(good_features).squeeze(2)
frames.shape

(83, 62, 2)

In [4]:
W = np.concatenate((U,V), axis=0)

In [5]:
W.shape

(166, 62)

In [6]:
W

array([[ 528.25891113,  524.25494385, 1366.23425293, ..., 1361.25024414,
         503.29119873, 1283.1697998 ],
       [ 527.92443848,  523.90515137, 1366.06445312, ..., 1361.05847168,
         503.02612305, 1283.02246094],
       [ 527.41503906,  523.41973877, 1365.68103027, ..., 1360.73522949,
         502.49887085, 1282.63000488],
       ...,
       [ 499.19107056,  505.43313599,  257.51379395, ...,  305.41717529,
         454.51022339,  395.85070801],
       [ 501.57394409,  507.9130249 ,  258.05737305, ...,  305.96243286,
         456.88049316,  396.76287842],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ]])

In [7]:
W.shape

(166, 62)

In [8]:
a_f = np.mean(W, axis=1).reshape(-1, 1)
a_f.shape

(166, 1)

In [9]:
W = W - a_f

In [10]:
U, S, V_t = np.linalg.svd(W)

In [11]:
U.shape, S.shape, V_t.shape

((166, 166), (62,), (62, 62))

In [12]:
U_ = U[:,:3]
S_ = np.diag(S[:3])
V_ = V_t[:3,:]

In [13]:
M_h = U_ # R*
S_h = S_ @ V_ # S*

In [14]:
Is = M_h[:n_frames, :]
Js = M_h[n_frames:, :]

In [15]:
func = lambda x, y: (x[0] * y[0],
                         x[0] * y[1] + x[1] * y[0],
                         x[0] * y[2] + x[2] * y[0],
                         x[1] * y[1],
                         x[1] * y[2] + x[2] * y[1],
                         x[2] * y[2])

G = np.zeros((3 * n_frames, 6))

In [16]:
G.shape,Is.shape,Js.shape

((249, 6), (83, 3), (83, 3))

In [17]:
for f in range(3 * n_frames):
    if f < n_frames:
        G[f, :] = func(Is[f, :], Is[f, :])
    elif f < 2 * n_frames:
        G[f, :] = func(Js[(f % (n_frames)), :],
                       Js[(f % (n_frames)), :])
    else:
        G[f, :] = func(Is[(f % (2 * n_frames)), :],
                       Js[(f % (2 * n_frames)), :])

c = np.concatenate((np.ones((2 * n_frames, 1)), np.zeros((n_frames, 1))))

In [18]:
l = np.linalg.lstsq(G, c)[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [19]:
def squarer(array):
    n = int(np.sqrt(array.size*2))
    R,C = np.triu_indices(n)
    out = np.zeros((n,n))
    out[R,C] = array
    out[C,R] = array
    return out

In [20]:
L = squarer(l.reshape(-1))

In [21]:
Q = np.linalg.cholesky(L)

M = M_h @ Q
S = np.linalg.inv(Q) @ S_h

In [22]:
M.shape

(166, 3)

In [23]:
S.shape

(3, 62)

In [27]:
cloud = PyntCloud(pd.DataFrame(
    data=np.hstack((S.T, 255*np.ones((S.T.shape[0],3)))),
    columns=["x", "y", "z", "red", "green", "blue"]))

cloud.to_file("../output/output.ply")
# Open using meshlab